В первом исследовании (select features.ipynb) на 100 пользователях для разных методов были найдены признаки, которые дают максимальную точность классификации. Лучшие результаты были получены с помощью методов:

метод                | средняя точность на контрольной | точность на тестовой
---------------------|---------------------------------|---------------------
KNeighborsClassifier | 0.93                            | 0.97
LogisticRegression   | 0.99                            | 1.0
SVC                  | 0.99                            | 0.99

Остальные методы проявили себя значительно слабее. Зная, какие признаки были выбраны для каждого их приведёных методов, можно выбрать наиболее часто встречающиеся признаки по каждому из методов и по всем методам и оценить точность классификации, но уже на полной выборке из 1220 пользователей.

In [1]:
knn_feat = [(3, 3), (4, 3), (6, 3), (11, 3), (2, 3), (1, 3), (13, 3), (10, 3), (5, 3), (12, 3), (7, 3), (28, 3), (9, 3), (0, 2), (27, 2), (19, 2), (14, 2), (8, 2), (15, 2), (29, 1), (18, 1), (23, 1), (33, 1), (36, 1), (37, 1), (41, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (34, 1), (20, 1)]
log_feat = [(27, 3), (7, 3), (4, 3), (2, 3), (3, 3), (0, 3), (8, 3), (9, 3), (6, 3), (13, 3), (12, 3), (1, 3), (11, 2), (10, 2), (5, 2), (28, 2), (36, 2), (15, 2), (19, 1), (32, 1), (25, 1), (21, 1), (20, 1), (35, 1), (14, 1)]
svc_feat = [(2, 3), (5, 3), (9, 3), (11, 3), (27, 3), (8, 3), (10, 3), (6, 3), (7, 3), (1, 3), (3, 3), (34, 3), (4, 3), (13, 3), (28, 2), (0, 2), (12, 2), (15, 2), (19, 1), (29, 1), (41, 1), (37, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (32, 1), (36, 1), (20, 1), (25, 1), (18, 1)]

Подготовим обучающую и тестовую выборки.

In [2]:
import numpy as np

train_set = np.load('train_set.npy')
test_set = np.load('test_set.npy')

X_train = np.nan_to_num(train_set[:, :-1])
X_test = np.nan_to_num(test_set[:, :-1])

y_train = train_set[:, -1].astype(int)
y_test = test_set[:, -1].astype(int)

In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Обучим каждую из моделей на признаках, которые встречаются 3 раза и 2 раза, и оценим точность на тестовой выборке.

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [11]:
def features(feat):
    count = 1
    
    while len(sorted([item[0] for item in feat if item[1] >= count])) > 0:
        yield sorted([item[0] for item in feat if item[1] >= count]), count
        count += 1

In [13]:
model = KNeighborsClassifier(n_neighbors=3)

for indexes, count in features(knn_feat):
    model.fit(X_train[:, indexes], y_train)
    print(model.score(X_test[:, indexes], y_test), 'при count', count)

0.8155737704918032 при count 1
0.8336065573770491 при count 2
0.8245901639344262 при count 3


In [15]:
model = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1_000)

for indexes, count in features(log_feat):
    model.fit(X_train[:, indexes], y_train)
    print(model.score(X_test[:, indexes], y_test), 'при count', count)

0.8967213114754098 при count 1
0.8844262295081967 при count 2
0.7959016393442623 при count 3


In [16]:
model = SVC(gamma='auto')

for indexes, count in features(svc_feat):
    model.fit(X_train[:, indexes], y_train)
    print(model.score(X_test[:, indexes], y_test), 'при count', count)

0.8704918032786885 при count 1
0.8934426229508197 при count 2
0.8704918032786885 при count 3


Наилучший результат показывают методы логистическая регрессия на признаках, которые встречаются 1 раз и чаще, и метод опорных векторов на признаках, которые встречаются 2 раза и чаще.

Проверим на совокупности признаков:

In [22]:
from collections import Counter
all_feat = Counter()

for item in knn_feat:
    all_feat[item[0]] += item[1]

for item in log_feat:
    all_feat[item[0]] += item[1]

for item in svc_feat:
    all_feat[item[0]] += item[1]

In [23]:
model = KNeighborsClassifier(n_neighbors=3)

for indexes, count in features(all_feat.most_common()):
    model.fit(X_train[:, indexes], y_train)
    print(model.score(X_test[:, indexes], y_test), 'при count', count)

0.7918032786885246 при count 1
0.8172131147540984 при count 2
0.8237704918032787 при count 3
0.830327868852459 при count 4
0.8336065573770491 при count 5
0.8336065573770491 при count 6
0.8418032786885246 при count 7
0.830327868852459 при count 8
0.6475409836065574 при count 9


In [24]:
model = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1_000)

for indexes, count in features(all_feat.most_common()):
    model.fit(X_train[:, indexes], y_train)
    print(model.score(X_test[:, indexes], y_test), 'при count', count)

0.9057377049180327 при count 1
0.9016393442622951 при count 2
0.8918032786885246 при count 3
0.8975409836065574 при count 4
0.8803278688524591 при count 5
0.8803278688524591 при count 6
0.8885245901639345 при count 7
0.8721311475409836 при count 8
0.5688524590163935 при count 9


In [25]:
model = SVC(gamma='auto')

for indexes, count in features(all_feat.most_common()):
    model.fit(X_train[:, indexes], y_train)
    print(model.score(X_test[:, indexes], y_test), 'при count', count)

0.8786885245901639 при count 1
0.8704918032786885 при count 2
0.8729508196721312 при count 3
0.8762295081967213 при count 4
0.8811475409836066 при count 5
0.8811475409836066 при count 6
0.8778688524590164 при count 7
0.8762295081967213 при count 8
0.7139344262295082 при count 9


На совокупности признаков лучшие результаты показали модели LogisticRegression и SVC. Логистическая регрессия показала точность 90% на тестовой выборке, но при этом использовались все признаки, что не очень хорошо, т.к. такая модель, вероятно, обладает меньшей обобщающей способностью. В то же время на признаках, которые встречаются 4 или 5 раз обе модели показали примерно равный результат - 88 - 89%. Вот соответствующие признаки:

In [30]:
for indexes, count in features(all_feat.most_common()):
    if count == 4 or count == 5:
        print(indexes)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 19, 27, 28, 34, 36]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 27, 28]


Далее, ипользуя только эти признаки, стоит выполнить настройку параметров моделей для максимального эффекта на кросс-валидации.

In [33]:
from sklearn.model_selection import GridSearchCV

In [35]:
indexes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 27, 28]

parameters = {
    'C'      : [0.01, 0.1, 1, 10],
    'solver' : ['newton-cg', 'lbfgs', 'sag', 'saga']}

estimator = GridSearchCV(LogisticRegression(), parameters, cv=3, n_jobs=-1)
estimator.fit(X_train[:, indexes], y_train)

C:\Users\stas\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'C': [0.01, 0.1, 1, 10], 'solver': ['newton-cg', 'lbfgs', 'sag', 'saga']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [36]:
print(estimator.best_estimator_)
print('best_score_', estimator.best_score_)
print('Точность на тестовой выборке', estimator.best_estimator_.score(X_test[:, indexes], y_test))

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)
best_score_ 0.8314207650273224
Точность на тестовой выборке 0.8663934426229508


In [40]:
parameters = {
    'C'      : [0.01, 0.1, 1, 10],
    'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree' : [2, 3, 4, 5],
    'gamma'  : ['auto', 'scale']}

estimator = GridSearchCV(SVC(), parameters, cv=3, n_jobs=-1)
estimator.fit(X_train[:, indexes], y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'C': [0.01, 0.1, 1, 10], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'degree': [2, 3, 4, 5], 'gamma': ['auto', 'scale']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [41]:
print(estimator.best_estimator_)
print('best_score_', estimator.best_score_)
print('Точность на тестовой выборке', estimator.best_estimator_.score(X_test[:, indexes], y_test))

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=2, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
best_score_ 0.8696721311475409
Точность на тестовой выборке 0.9040983606557377
